In [1]:
# Import required modules
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib as plt

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [2]:
#import training data set
train = pd.read_csv("../data/raw/train.csv.zip", compression="zip", index_col='id')
meta_data = pd.read_csv("../data/interim/training_metadata.csv")

In [3]:
# Names
cat_names = meta_data[meta_data['Data Type']=='object']['Variable']
cont_names = meta_data[(meta_data['Data Type']=='float64') & 
                       (meta_data['Variable']!='loss')]['Variable']

# Merge transformed categorical data with continuous
cont_names_to_drop =['cont12' # highly correlated to cont11
                     ,'cont9' # highly correlated to cont1
                    ]
cont_names_final = cont_names[~cont_names.isin(cont_names_to_drop)]

In [4]:
import category_encoders as ce
encoder = ce.BinaryEncoder()

categorical_data = encoder.fit_transform(train[cat_names])
continuous_data = train[cont_names]

train_final1 = pd.concat([train[cont_names], categorical_data], axis=1)

C:\Users\ryan.shuhart\AppData\Local\Continuum\Anaconda3\lib\site-packages\category_encoders\ordinal.py:190: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  X[col] = X[col].astype(int).reshape(-1, )
C:\Users\ryan.shuhart\AppData\Local\Continuum\Anaconda3\lib\site-packages\category_encoders\ordinal.py:179: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  X[switch.get('col')] = X[switch.get('col')].astype(int).reshape(-1, )


In [5]:
train_final1 = pd.concat([train[cont_names], categorical_data], axis=1)
X=train_final1.values
y=train['loss'].values

# SGD Model

In [26]:
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.grid_search import GridSearchCV

scorer = make_scorer(mean_absolute_error)
clf = linear_model.SGDRegressor()

scores = cross_val_score(clf, X, y, cv=10, scoring=scorer)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 1328.66 (+/- 62.59)


In [29]:
param_grid = [{'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
               'penalty':['none', 'l2', 'l1', 'elasticnet']
              }]

gs = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, scoring=scorer)
gs = gs.fit(X, y)

print("Best F-Score: ", gs.best_score_)
print("Best Parameters: ", gs.best_params_)
print("Best Estimator: ", gs.best_estimator_)

Best F-Score:  1893.1984383073209
Best Parameters:  {'penalty': 'l2', 'loss': 'huber'}
Best Estimator:  SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='huber', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)


## TPOT

In [32]:
from tpot import TPOTRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.6, test_size=0.4)

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('../models/tpot_pipeline.py')

Optimization Progress:   2%|▏         | 2/120 [05:19<3:24:22, 103.92s/pipeline]


GP closed prematurely - will use current best pipeline


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
ValueError: A pipeline has not yet been optimized. Please call fit() first.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
AssertionError


ValueError: A pipeline has not yet been optimized. Please call fit() first.